# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "output.csv"
csv = "output.csv"
weather_data_df = pd.read_csv("output.csv")
weather_data_df

,city,lat,long,temp,humidity,cloud,wind,country,date and time
0,butaritari,3.0707,172.7902,80.76,81,72,12.37,KI,1615657191
1,aklavik,68.2191,-135.0107,-22.00,76,20,3.44,CA,1615657442
2,cayenne,4.9333,-52.3333,84.27,62,40,11.50,GF,1615657196
3,galveston,29.3669,-94.9669,72.28,78,75,11.50,US,1615657443
4,kavieng,-2.5744,150.7967,81.30,65,0,4.81,PG,1615657443
...,...,...,...,...,...,...,...,...,...
563,lerwick,60.1545,-1.1494,42.80,87,75,17.27,GB,1615657630
564,ruteng,-8.6136,120.4721,62.40,91,95,0.31,ID,1615657631
565,taunggyi,20.7833,97.0333,61.09,35,0,1.59,MM,1615657631
566,presidencia roque saenz pena,-26.7852,-60.4388,103.35,16,80,2.30,AR,1615657631


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

coordinates =weather_data_df[["lat", "long"]]
humidity = weather_data_df["humidity"].astype(float)

fig = gmaps.figure()
figure_layout = {
    'width': '400px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto',
    'zoom' : '8x'
}

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
vacation_df = weather_data_df
# Create the names and bins for temperature
temp_bins = [-100,70, 80, 400]
temp_group_names = ["too cool", "70-80", "too hot"]

# Create the names and bins for wind
wind_bins = [-5, 10, 500]
wind_group_names = ["less than 10 mph", "more than 10 mph"]

# Create the names and bins for cloudiness
cloud_bins = [-5, 0.1, 500]
cloud_group_names = ["clear", "cloudy"]


vacation_df["temp"] = pd.cut(vacation_df["temp"], temp_bins, labels=temp_group_names, include_lowest=True)
vacation_df["wind"] = pd.cut(vacation_df["wind"], wind_bins, labels=wind_group_names, include_lowest=True)
vacation_df["cloud"] = pd.cut(vacation_df["cloud"], cloud_bins, labels=cloud_group_names, include_lowest=True)
vacation_df

,city,lat,long,temp,humidity,cloud,wind,country,date and time
0,butaritari,3.0707,172.7902,too hot,81,cloudy,more than 10 mph,KI,1615657191
1,aklavik,68.2191,-135.0107,too cool,76,cloudy,less than 10 mph,CA,1615657442
2,cayenne,4.9333,-52.3333,too hot,62,cloudy,more than 10 mph,GF,1615657196
3,galveston,29.3669,-94.9669,70-80,78,cloudy,more than 10 mph,US,1615657443
4,kavieng,-2.5744,150.7967,too hot,65,clear,less than 10 mph,PG,1615657443
...,...,...,...,...,...,...,...,...,...
563,lerwick,60.1545,-1.1494,too cool,87,cloudy,more than 10 mph,GB,1615657630
564,ruteng,-8.6136,120.4721,too cool,91,cloudy,less than 10 mph,ID,1615657631
565,taunggyi,20.7833,97.0333,too cool,35,clear,less than 10 mph,MM,1615657631
566,presidencia roque saenz pena,-26.7852,-60.4388,too hot,16,cloudy,less than 10 mph,AR,1615657631


In [18]:
ideal_vacation_df=vacation_df.loc[vacation_df["temp"]=="70-80"]
ideal_vacation_df=ideal_vacation_df.loc[ideal_vacation_df["wind"]=="less than 10 mph"]
ideal_vacation_df=ideal_vacation_df.loc[ideal_vacation_df["cloud"]=="clear"]
hotel_df=ideal_vacation_df.loc[ideal_vacation_df["humidity"]<75]
hotel_df

#hotel_df =ideal_vacation_df.loc[ideal_vacation_df["country"]=="IN"].copy()
#hotel_df

,city,lat,long,temp,humidity,cloud,wind,country,date and time
32,salalah,17.0151,54.0924,70-80,69,clear,less than 10 mph,OM,1615657451
98,mar del plata,-38.0023,-57.5575,70-80,41,clear,less than 10 mph,AR,1615657469
117,port elizabeth,-33.9180,25.5701,70-80,73,clear,less than 10 mph,ZA,1615657474
388,myanaung,18.2833,95.3167,70-80,37,clear,less than 10 mph,MM,1615657566
448,mangrol,21.1167,70.1167,70-80,74,clear,less than 10 mph,IN,1615657588
458,mul,20.0667,79.6667,70-80,23,clear,less than 10 mph,IN,1615657592
474,giyani,-23.3025,30.7187,70-80,62,clear,less than 10 mph,ZA,1615657598


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel = []
locations = hotel_df["city"]
# geocoordinates

target_radius = 5000
target_type = "lodging"

for index,row in hotel_df.iterrows():
    target_coordinates = f'{row["lat"]},{row["long"]}'
# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary

    
    response = requests.get(base_url, params=params)
#hotel.append(response['hotel'])
    #print(response.json())
    hotel_response=response.json()
    hotel.append(hotel_response["results"][0]["name"])
hotel_df["Hotel Name"]=""
hotel_df.loc[:,"Hotel Name"]=hotel

hotel_df

<ipython-input-19-be7c5a748397>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""
C:\Users\brand\.conda\envs\pygmaps\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,city,lat,long,temp,humidity,cloud,wind,country,date and time,Hotel Name
32,salalah,17.0151,54.0924,70-80,69,clear,less than 10 mph,OM,1615657451,HAMDAN PLAZA HOTEL SALALAH
98,mar del plata,-38.0023,-57.5575,70-80,41,clear,less than 10 mph,AR,1615657469,Gran Hotel Mar del Plata
117,port elizabeth,-33.9180,25.5701,70-80,73,clear,less than 10 mph,ZA,1615657474,39 On Nile Guest House
388,myanaung,18.2833,95.3167,70-80,37,clear,less than 10 mph,MM,1615657566,Taung Hlay Khar
448,mangrol,21.1167,70.1167,70-80,74,clear,less than 10 mph,IN,1615657588,Hotel The Royal Honours
458,mul,20.0667,79.6667,70-80,23,clear,less than 10 mph,IN,1615657592,Taj Mahal
474,giyani,-23.3025,30.7187,70-80,62,clear,less than 10 mph,ZA,1615657598,Kulani Country Lodge


In [20]:
hotel_df=hotel_df.rename(columns={"city":"City", "country":"Country", "lat":"Lat", "long":"Lng"})


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)
fig
                             
# Display figure


Figure(layout=FigureLayout(height='420px'))